<a href="https://colab.research.google.com/github/ShivaniGawande/DLFinalProject_Fall22/blob/main/ReadingBetweenLines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Information about data, project and contents of the notebook

# Data Loading

Importing Libraries

Loading Dataset

# Exploratory Data Analysis

Meta Features 

Embeddings and Data Cleaning

Handling Mislabeled Samples

#Model Implementation

Baseline model - word2vec

BERT

CLIP + cosine similarity

#Validation 

Plot for Word2Vec (Baseline)

Plot for BERT

Plot for CLIP + cosine similarity

#Comparing all the three models

Inference